In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!nohup ollama serve &
!sleep 8 && tail nohup.out

nohup: appending output to 'nohup.out'
[GIN-debug] GET    /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] GET    /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] GET    /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
[GIN-debug] HEAD   /                         --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func1 (5 handlers)
[GIN-debug] HEAD   /api/tags                 --> github.com/ollama/ollama/server.(*Server).ListHandler-fm (5 handlers)
[GIN-debug] HEAD   /api/version              --> github.com/ollama/ollama/server.(*Server).GenerateRoutes.func2 (5 handlers)
time=2024-12-23T13:57:51.408Z level=INFO source=routes.go:1310 msg="Listening on 127.0.0.1:11434 (version 0.5.4)"
time=2024-12-23T13:57:51.409Z level=INFO source=routes.go:1339 msg="Dynamic LLM libraries" runners="[cuda_v11

In [3]:
!ollama pull llama3
!ollama pull mxbai-embed-large


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏ 6.9 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   1% ▕▏  66 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   2% ▕▏ 100 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   3% ▕▏ 147 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   4% ▕▏ 188 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 230 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   5% ▕▏ 249 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   7% ▕▏ 303 MB/4.7 GB  303 MB/s   

In [4]:
!pip install -r requirements.txt
!pip install tokenizers

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of transformers to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 43.4 MB/s e

In [1]:
%%writefile constants.py
import os
from chromadb.config import Settings

# Define the folder for storing database
PERSIST_DIRECTORY = os.environ.get('PERSIST_DIRECTORY', 'db')

# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)


Writing constants.py


In [6]:
import os
import csv
import uuid
import shutil
import glob
import re
import asyncio
from datetime import datetime
from typing import List, Dict

from pydantic import BaseModel
from jinja2 import Template
from langchain_ollama import OllamaLLM
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import CrossEncoder
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker
from langchain.docstore.document import Document
import warnings
warnings.filterwarnings("ignore")





In [7]:

PERSIST_DIRECTORY = os.environ.get('PERSIST_DIRECTORY', 'db')

CHROMA_SETTINGS = {
    "persist_directory": PERSIST_DIRECTORY,
    "anonymized_telemetry": False
}

SOURCE_DIRECTORY = os.environ.get('SOURCE_DIRECTORY', 'source_documents')
CHUNK_SIZE = 500
CHUNK_OVERLAP = 50
OVERVIEW_FILEPATH = os.path.join('data', 'overview.csv')


In [8]:
class Bot:
    def __init__(self) -> None:
        self.bot_id = str(uuid.uuid4())
        self.source_dir = os.path.join('data', 'source_documents', self.bot_id)
        self.db_dir = os.path.join('data', 'vector_db', self.bot_id)
        self.create_dir(self.source_dir)
        self.create_dir(self.db_dir)

    @staticmethod
    def create_dir(directory: str):
        if os.path.exists(directory):
            shutil.rmtree(directory)
        os.makedirs(directory)


In [9]:
def get_db_dir(bot_id: str) -> str:
    return os.path.join('data', 'vector_db', bot_id)


In [10]:
class CustomTemplates:
    def __init__(self, model: str) -> None:
        models = ['llama3']
        if model not in models:
            raise RuntimeError('No Models Found')
        self.active = models.index(model)
        if self.active == 0:
            self.history = Template(for_llama3_history)

    def render_template(self, content: List[Dict[str, str]]) -> str:
        if self.active == 0:
            return self.history.render({"messages": content})
        # Extend for more models if needed

# Define the Jinja2 template string
for_llama3_history = """
{% for message in messages %}
    {% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}
        {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}
    {% endif %}
    {% if message['role'] == 'user' %}
        USER: {{ message['content'] }}
    {% elif message['role'] == 'assistant' %}
        MACHINE: {{ message['content'] }}
    {% else %}
        {{ raise_exception('Only user and assistant roles are supported!') }}
    {% endif %}
{% endfor %}
"""
for_llama3_route_query = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You help triage user request, \
    do NOT answer the question itself, do NOT explain yourself. \
    Given a text input, output either DOCS or DEFAULT, according to these definitions:

    DOCS: if user is asking an ambiguous question that seems to require knowledge from some external documentation
    DEFAULT: if user is just chit-chatting

    Output ONE word answer: DOCS or DEFAULT, WITHOUT explanation. <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, what is your question <|eot_id|><|start_header_id|>user<|end_header_id|>
    {query} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

for_llama3_sort_relevancy = """<|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>\
    You help classify the relevancy of the context info to the question, \
    do NOT answer the question, do NOT explain yourself. \
    Given some documents, and a question from human, output either YES or NO, by the following definitions:

    YES: if the documents CAN help you derive an answer for the question
    NO: if the documents is NOT relevant to the question

    Output ONE word answer: YES, or NO, WITHOUT explanation. <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, give me the documents <|eot_id|><|start_header_id|>user<|end_header_id|>
    DOCUMENTS: {context} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, what is your question <|eot_id|><|start_header_id|>user<|end_header_id|>
    {query} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

for_llama3_qa_from_docs = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You help answer humans's questions, only based on the given documents. \
    Given some documents, and a human question, output your answer. Keep it short. <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, give me the documents <|eot_id|><|start_header_id|>user<|end_header_id|>
    DOCUMENTS: {context} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, what is your question <|eot_id|><|start_header_id|>user<|end_header_id|>
    {query} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

for_llama3_contextualize_in_history = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    Given a chat history and the most recent question, you help formulate the question into \
    a standalone question that can be understood without the chat history. \
    Do NOT answer the question, just reformulate it if needed and otherwise return it as is. \
    Output only that formulated question. <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, tell me the chat history. Use USER:, and MACHINE: to show whose turn it is in the history <|eot_id|><|start_header_id|>user<|end_header_id|>
    CHAT HISTORY: {chat_history} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, what is your most recent question that needs formulated? <|eot_id|><|start_header_id|>user<|end_header_id|>
    {query} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

for_llama3_clarify = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You help gather more information. \
    Given some documents, and a human question, which is not directly relevant to the documents, \
    do NOT answer the question, but ask a clarification question, for users to elaborate on question question. \
    Tell the user that the question needs elaboration by ouputing one single clarification question. \
    <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, give me the documents <|eot_id|><|start_header_id|>user<|end_header_id|>
    DOCUMENTS: {context} <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    Ok, what is your question <|eot_id|><|start_header_id|>user<|end_header_id|>
    {query} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"""


In [11]:

class Prompts:
    def __init__(self, model: str) -> None:
        models = ['llama3']
        if model not in models:
            raise RuntimeError('No Models Found')
        self.active = models.index(model)
        if self.active == 0:
            self.route_query = for_llama3_route_query
            self.sort_relevancy = for_llama3_sort_relevancy
            self.qa_from_docs = for_llama3_qa_from_docs
            self.contextualize_in_history = for_llama3_contextualize_in_history
            self.clarify = for_llama3_clarify


In [13]:
from langchain_community.document_loaders import (
    CSVLoader,
    EverNoteLoader,
    PyMuPDFLoader,
    TextLoader,
    UnstructuredEmailLoader,
    UnstructuredEPubLoader,
    UnstructuredHTMLLoader,
    UnstructuredMarkdownLoader,
    UnstructuredODTLoader,
    UnstructuredPowerPointLoader,
    UnstructuredWordDocumentLoader,
)

from constants import CHROMA_SETTINGS

LOADER_MAPPING = {
    ".csv": (CSVLoader, {}),
    ".doc": (UnstructuredWordDocumentLoader, {}),
    ".docx": (UnstructuredWordDocumentLoader, {}),
    ".html": (UnstructuredHTMLLoader, {}),
    ".md": (UnstructuredMarkdownLoader, {}),
    ".odt": (UnstructuredODTLoader, {}),
    ".pdf": (PyMuPDFLoader, {}),
    ".ppt": (UnstructuredPowerPointLoader, {}),
    ".pptx": (UnstructuredPowerPointLoader, {}),
    ".txt": (TextLoader, {"encoding": "utf8"}),
}


async def write_file(file_content: bytes, file_path: str):
    async with aiofiles.open(file_path, 'wb') as out_file:
        await out_file.write(file_content)

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^\w\s\'\".,?@:/]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    dates = re.findall(r'\d{1,2}/\d{1,2}/\d{4}', text)
    for date in dates:
        standardized_date = datetime.strptime(date, '%m/%d/%Y').strftime('%Y-%m-%d')
        text = text.replace(date, standardized_date)
    return text

def load_single_document(file_path: str) -> List[Document]:
    ext = "." + file_path.rsplit(".", 1)[-1]
    if ext in LOADER_MAPPING:
        loader_class, loader_args = LOADER_MAPPING[ext]
        loader = loader_class(file_path, **loader_args)
        docs = loader.load()
        if ext in ['.pdf', '.html', '.txt']:
            for doc in docs:
                doc.page_content = clean_text(doc.page_content)
        return docs
    raise ValueError(f"Unsupported file extension '{ext}'")

def load_documents(source_dir: str, ignored_files: List[str] = []) -> List[Document]:
    all_files = []
    for ext in LOADER_MAPPING:
        all_files.extend(
            glob.glob(os.path.join(source_dir, f"**/*{ext}"), recursive=True)
        )
    documents = []
    for file_path in all_files:
        if file_path in ignored_files:
            continue
        documents.extend(load_single_document(file_path))
        print(f"Loaded document {file_path}")
    return documents

def process_documents(source_folder: str, embedding_model=None, ignored_files: List[str] = []) -> List[Document]:
    print(f"Loading documents from {source_folder}")
    documents = load_documents(source_folder, ignored_files)
    if not documents:
        print("No new documents to load")
        return []
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=CHUNK_SIZE, chunk_overlap=CHUNK_OVERLAP)
    if embedding_model:
        text_splitter_semantic = SemanticChunker(embedding_model)
        documents = text_splitter_semantic.split_documents(documents)
    texts = text_splitter.split_documents(documents)
    print(f"Split into {len(texts)} chunks of text (max. {CHUNK_SIZE} tokens each)")
    return texts

def does_vectorstore_exist(persist_directory: str) -> bool:
    if os.path.exists(os.path.join(persist_directory, 'index')):
        if os.path.exists(os.path.join(persist_directory, 'chroma-collections.parquet')) and \
           os.path.exists(os.path.join(persist_directory, 'chroma-embeddings.parquet')):
            list_index_files = glob.glob(os.path.join(persist_directory, 'index/*.bin'))
            list_index_files += glob.glob(os.path.join(persist_directory, 'index/*.pkl'))
            if len(list_index_files) > 3:
                return True
    return False

def ingest(db_folder: str, source_folder: str):
    embedding_model = OllamaEmbeddings(
        model="mxbai-embed-large",
        # base_url="http://host.docker.internal:11434"
    )
    if does_vectorstore_exist(db_folder):
        print(f"Appending to existing vectorstore at {db_folder}")
        db = Chroma(persist_directory=db_folder, embedding_function=embedding_model, client_settings=CHROMA_SETTINGS)
        collection = db.get()
        texts = process_documents(source_folder, embedding_model=embedding_model, ignored_files=[metadata['source'] for metadata in collection['metadatas']])
        print(f"Creating embeddings. May take some minutes...")
        db.add_documents(texts)
    else:
        print("Creating new vectorstore")
        texts = process_documents(source_folder, embedding_model=embedding_model)
        print(f"Creating embeddings. May take some minutes...")
        db = Chroma.from_documents(texts, embedding_model, persist_directory=db_folder)
    db.persist()
    db = None


In [14]:
class Agent(BaseModel):
    bot_id: str
    name: str
    description: str
    starter: str
    model: str

session_cache = {}
embedding_model = OllamaEmbeddings(
    model="mxbai-embed-large",
    # base_url="http://host.docker.internal:11434"
)

def load_model(model_name: str) -> OllamaLLM:
    return OllamaLLM(
        model=model_name,
        device='cpu',
        callbacks=[StreamingStdOutCallbackHandler()],
    )

def load_retriever(bot_id: str) -> Chroma:
    return Chroma(
        persist_directory=get_db_dir(bot_id),
        embedding_function=embedding_model
    ).as_retriever(search_kwargs={"k": 10})

def rerank_docs(reranker_model: CrossEncoder, query: str, retrieved_docs: List[Document]) -> List[tuple]:
    query_and_docs = [(query, r.page_content) for r in retrieved_docs]
    scores = reranker_model.predict(query_and_docs)
    return sorted(list(zip(retrieved_docs, scores)), key=lambda x: x[1], reverse=True)

def load_agent(agent: Agent):
    session_cache[agent.bot_id] = {
        'name': agent.name,
        'description': agent.description,
        'starter': agent.starter,
        'llm': load_model(agent.model),
        'retriever': load_retriever(agent.bot_id),
        'reranker': CrossEncoder("mixedbread-ai/mxbai-rerank-large-v1", device='cpu'),
        'history': [],
        'prompts': Prompts(agent.model),
        'jinja_templates': CustomTemplates(agent.model)
    }

def find_bot_by_id(bot_id: str) -> Agent:
    if not os.path.exists(OVERVIEW_FILEPATH):
        return None
    with open(OVERVIEW_FILEPATH, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['bot_id'] == bot_id:
                return Agent(**row)
    return None

async def on_message(bot_id: str, content: str) -> Dict[str, List[str]]:
    print('\nBEGIN PROCESS\n')
    if bot_id not in session_cache:
        agent = find_bot_by_id(bot_id)
        if agent:
            await load_agent(agent)
        else:
            raise ValueError(f"Bot with id {bot_id} does not exist")

    bot_info = session_cache[bot_id]
    llm = bot_info['llm']
    retriever = bot_info["retriever"]
    reranker = bot_info["reranker"]
    history = bot_info['history']
    prompts = bot_info['prompts']
    jinja_templates = bot_info['jinja_templates']

    if len(history) > 0:
        rendered_history = jinja_templates.render_template(history)
        content = llm.invoke(prompts.contextualize_in_history.format(chat_history=rendered_history, query=content), stop=['<|eot_id|>'])
        print('\nDONE CONTEXTUALIZING\n')

    source_documents = None
    route = llm.invoke(prompts.route_query.format(query=content), stop=['<|eot_id|>'])
    print('\nDONE ROUTING\n')
    if route.strip().startswith('DOCS'):
        retrieved_docs = retriever.get_relevant_documents(f"Represent this sentence for searching relevant passages: {content}")
        print('\nDONE RETRIEVED\n')

        reranked_docs = rerank_docs(reranker, content, retrieved_docs)
        print('\nDONE RERANKING\n')

        source_documents = [doc[0] for doc in reranked_docs[:4]]
        docs_content = '\n'.join([doc.page_content for doc in source_documents])

        relevancy = llm.invoke(prompts.sort_relevancy.format(context=docs_content, query=content), stop=['<|eot_id|>'])
        print('\nDONE RELEVANCY\n')
        if relevancy.strip().startswith('YES'):
            answer = llm.invoke(prompts.qa_from_docs.format(context=docs_content, query=content), stop=['<|eot_id|>'])
        else:
            answer = llm.invoke(prompts.clarify.format(context=docs_content, query=content), stop=['<|eot_id|>'])
    else:
        answer = llm.invoke(content, stop=['<|eot_id|>'])

    history.append({"role": "user", "content": content})
    history.append({"role": "assistant", "content": answer})
    bot_info['history'] = history[-8:]

    sources = [doc.page_content for doc in source_documents] if source_documents else []
    return {'response': answer, 'sources': sources}


In [15]:
def overview() -> List[Agent]:
    agents = []
    if not os.path.exists(OVERVIEW_FILEPATH):
        return agents
    with open(OVERVIEW_FILEPATH, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            agents.append(Agent(**row))
    return agents


In [16]:

async def create_agent(agent: Agent):
    # Load agent into session_cache
    await load_agent(agent)
    # Append agent details to overview.csv
    with open(OVERVIEW_FILEPATH, mode='a', newline='') as file:
        writer = csv.writer(file)
        # Check if the file is empty to write headers
        if file.tell() == 0:
            writer.writerow(['bot_id', 'name', 'description', 'starter', 'model'])
        writer.writerow([agent.bot_id, agent.name, agent.description, agent.starter, agent.model])

def preview_agent(agent: Agent) -> str:
    if agent.bot_id not in session_cache:
        asyncio.run(load_agent(agent))
    session_cache[agent.bot_id]['history'] = []
    return session_cache[agent.bot_id]['starter']


In [17]:
def upload_documents(bot: Bot, file_paths: List[str]):
    for file_path in file_paths:
        filename = os.path.basename(file_path)
        dest_path = os.path.join(bot.source_dir, filename)
        shutil.copy(file_path, dest_path)
    print("Uploading Complete")
    ingest(bot.db_dir, bot.source_dir)
    print("Ingestion Complete")


In [18]:
# Example usage
bot = Bot()
file_paths = ['attentionisyouallyouneed.pdf']
upload_documents(bot, file_paths)


Uploading Complete
Creating new vectorstore
Loading documents from data/source_documents/a6f2c431-b4ed-4c8d-b818-9b5f4ae11946
Loaded document data/source_documents/a6f2c431-b4ed-4c8d-b818-9b5f4ae11946/attentionisyouallyouneed.pdf
Split into 108 chunks of text (max. 500 tokens each)
Creating embeddings. May take some minutes...
Ingestion Complete


In [19]:
new_agent = Agent(
    bot_id=bot.bot_id,
    name="Example Bot",
    description="Rag agent for Corpotatica.",
    starter="Hello! How can I assist you today?",
    model="llama3"
)

create_agent(new_agent)
print(f"Agent {new_agent.name} created with ID: {new_agent.bot_id}")


Agent Example Bot created with ID: a6f2c431-b4ed-4c8d-b818-9b5f4ae11946


In [21]:
starter_message = preview_agent(new_agent)
print(starter_message)


Hello! How can I assist you today?


In [22]:
user_inputs = ['What is the key innovation of the Transformer architecture compared to previous sequence transduction models?',
              'How does the Transformer achieve faster training times compared to RNN-based models?',
              'What is the purpose of scaling the dot products by 1/√dk in the attention mechanism?',
              "What modifications were needed in the decoder's self-attention layers to preserve the auto-regressive property?"]

for user_input in user_inputs:
  response = asyncio.run(on_message(new_agent.bot_id, user_input))
  print("Bot Response:", response['response'])
  print("Sources:", response['sources'])



BEGIN PROCESS

DOCS
DONE ROUTING


DONE RETRIEVED


DONE RERANKING

YES
DONE RELEVANCY

According to the documents, the key innovation of the Transformer architecture is that it relies entirely on attention mechanisms to draw global dependencies between input and output sequences, dispensing with recurrence or convolution.Bot Response: According to the documents, the key innovation of the Transformer architecture is that it relies entirely on attention mechanisms to draw global dependencies between input and output sequences, dispensing with recurrence or convolution.
Sources: ['performs sur prisingly well, yielding better results than all previously reported models with the exception of the recurrent neural network grammar 8. in contrast to rnn sequencetosequence models 37, the transformer outperforms the berkeley parser 29 even when training only on the wsj training set of 40k sentences. 7 conclusion in this work, we presented the transformer, the first sequence transduction model b